# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import numpy and pandas
import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [3]:
# Run this code:

salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [4]:
# Your code here:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
# Your code here:
salaries.isnull().sum()*100/len(salaries)

Name                  0.000000
Job Titles            0.000000
Department            0.000000
Full or Part-Time     0.000000
Salary or Hourly      0.000000
Typical Hours        75.824971
Annual Salary        24.175029
Hourly Rate          75.824971
dtype: float64

In [6]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [7]:
# Your code here:
salaries['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [8]:
# Your code here:
salaries.groupby('Department').count().Name

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [9]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [13]:
import numpy as np
from scipy import stats
import statsmodels as sm

In [89]:
df_hourly_rate = salaries.get(salaries['Hourly Rate'].notnull())
stats, pvalue = sm.stats.weightstats.ztest(df_hourly_rate['Hourly Rate'], None, value = 30) 
print(f'The z score is: {stats:.2f} and the p value is: {pvalue}')  #z score es como de lejos está de la media. 

The z score is: 20.62 and the p value is: 1.8228873859286195e-94


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [33]:
# Your code here:
df_annualy = salaries.get(salaries['Annual Salary'].notnull())
stats, pvalue = sm.stats.weightstats.ztest(df_annualy['Annual Salary'], None, value = 86000) 
print(f'The z score is: {stats:.2f} and the p value is: {pvalue}')  #z score es como de lejos está de la media. 

The z score is: 5.93 and the p value is: 2.9768361642669956e-09


Using the `crosstab` function, find the department that has the most hourly workers. 

In [18]:
# Your code here:
pd.crosstab(salaries['Department'], salaries['Salary or Hourly']).sort_values(salaries['Salary or Hourly'].min(), ascending = False)


Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215
TRANSPORTN,725,415
PUBLIC LIBRARY,299,716
FAMILY & SUPPORT,287,328
CITY COUNCIL,64,347


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [76]:
# Your code here:
from scipy import stats
# H0 >= 35
# H1 < 35
stats, pvalue = stats.ttest_1samp(df_hourly['Hourly Rate'], 35)

print(f'stats: {stats:.2f}, pvalue: {pvalue}')

stats: -16.35, pvalue: 3.7636889298508017e-59


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [57]:
from scipy.stats import sem, t

In [94]:
# Your code here: 
df_hourly = salaries.get(salaries['Salary or Hourly'] == 'Hourly')
confidence = 0.95
hourly_wage = df_hourly['Hourly Rate']

n = len(hourly_wage)
m = np.mean(hourly_wage)
std = np.std(hourly_wage)

h = std * t(n-1).ppf((1 + confidence) / 2)

start = m - h
end = m + h 

print(f'The confidence interval goes from {m - h:.2f} to {m + h :.2f}')
print(f'The standard error is: {sem(hourly_wage):.2f}')

The confidence interval goes from 9.05 to 56.53
The standard error is: 0.14


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [101]:
# Your code here:
df_salary = salaries.get(salaries['Salary or Hourly'] == 'Salary')
confidence = 0.95
salaried = df_salary['Annual Salary']

n = len(salaried)
m = np.mean(salaried)
std = np.std(salaried)

h = std * t(n-1).ppf((1 + confidence) / 2)

start = m - h
end = m + h 

print(f'The confidence interval goes from {m - h:.2f} to {m + h :.2f}')
print(f'The standard error is: {sem(salaried):.2f}')

The confidence interval goes from 45545.54 to 128028.46
The standard error is: 132.65


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [23]:
from statsmodels.stats.proportion import proportions_ztest

In [46]:
# Your code here:
#h0 = 0.25
#h1 ≠ 0.25
count = salaries['Hourly Rate'].count()
nobs = len(salaries)
value = 0.25

stat, pval = proportions_ztest(count, nobs, value)
print('{0:0.10f}'.format(pval))

0.0004481127
